# Introducing Colab

Learn about Google Colab and set up a Google account (if you don't already have one) [here](https://colab.research.google.com/)

You should be able to use the free tier or minimal spend to complete all the projects in the class. I personally signed up for Colab Pro+ and I'm loving it - but it's not required.

To see some of what Colab can do, please follow the link below!

https://colab.research.google.com/drive/1DjcrYDZldAXKJ08x1uYIVCtItoLPk1Wr?usp=sharing

In [16]:
from google.colab import userdata
from huggingface_hub import login
from IPython.display import display, Audio
from transformers import pipeline
import soundfile as sf
from diffusers import AutoPipelineForText2Image, DiffusionPipeline
import torch
import IPython

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


In [17]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

TimeoutException: Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print("No GPU found")
else:
    print(gpu_info)
    if gpu_info.find('Tesla T4') >= 0:
        print("You have a Tesla T4 GPU")
    else:
        print("You don't have a Tesla T4 GPU")

Thu Jan 15 20:58:28 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
pipe = AutoPipelineForText2Image.from_pretrained('stabilityai/sdxl-turbo', torch_dtype=torch.float16, variant='fp16')
pipe.to('cuda')
prompt = 'A class of students learning AI engineering in a vibrant pop-art style'
image = pipe(prompt=prompt, num_inference_steps=4, guidance_scale=0.0).images[0]
display(image)

In [ ]:
# IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
base = DiffusionPipeline.from_pretrained('stabilityai/stable-diffusion-xl-base-1.0', torch_dtype=torch.float16, variant='fp16')
base.to('cuda')
refiner = DiffusionPipeline.from_pretrained('stabilityai/stable-diffusion-xl-refiner-1.0', torch_dtype=torch.float16, variant='fp16')
refiner.to('cuda')

n_steps = 40
high_noise_frac = 0.8

prompt = "A fantasy landscape, trending on artstation"

image = base(
    prompt=prompt,
    num_inference_steps=int(n_steps * high_noise_frac),
    denoising_end=high_noise_frac,
    output_type='latent'
).images

image = refiner(
    prompt=prompt,
    image=image,
    num_inference_steps=int(n_steps * (1 - high_noise_frac)),
    denoising_start=high_noise_frac,
).images[0]

display(image)

NameError: name 'DiffusionPipeline' is not defined

In [ ]:
# IPython.Application.instance().kernel.do_shutdown(True)

In [18]:
synthesizer = pipeline('text-to-speech', model='microsoft/speecht5_tts', device='cuda')
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-embeddings", split="validation", trust_remote_code=True)
speaker_embedding = torch.tensor(embeddings_dataset[7306]['xvector']).unsqueeze(0)
speech = synthesizer('Hi to an artificial intelligence engineer, on the way to mastery!', forward_params={'speaker_embeddings': speaker_embedding}, return_tensors=True, sampling_rate=22050)

Audio(speech['audio'], rate=speech['sampling_rate'])

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/585M [00:00<?, ?B/s]

...

Device set to use cuda


config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/50.7M [00:00<?, ?B/s]

NameError: name 'load_dataset' is not defined

model.safetensors:   0%|          | 0.00/50.6M [00:00<?, ?B/s]